<b style="font-size:2em;">Final Project Submission</b>

* Student name: Edward Amor 
* Student pace: part time
* Scheduled project review date/time: 
* Instructor name: Victor Geislinger
* Blog post URL: 

A real estate company in the King County area wants to investigate house sales in order to predict pricing. Our task is to clean, explore, and model the housing data they have supplied to us in `kc_house_data.csv`, and identify house features which significantly effect house price. 

# Business Understanding

## Background

A real estate company in the King County area wants to investigate house sales in order to predict pricing. Our task is to clean, explore, and model the housing data they have supplied to us in `kc_house_data.csv`, and identify house features which significantly effect house price. 

## Scope

To identify which house features significantly effect house price, we will use standard EDA practices, hypothesis testing, and regression modeling to come to our conclusions.

For our client's consumption the results of our analysis will be condensed into a presentation. 

## Goal

Our goal is to generate a multivariate linear regression model which as accurately as possible predicts the sale price of houses.  

## Objectives

1. Data Acquisition
2. Data Understanding
  - Data Cleaning
  - Data Exploration
  - Data Visualization
3. Modeling
  - Feature Engineering
    - Feature Transformation
    - Feature Selection
  - Training
  - Model Evaluation